# Run PncA WandB Sweep

In [1]:
from IPython.display import display
import os

if "SSH_CONNECTION" in os.environ:
    display("Running via SSH")
else:
    display("Running locally")
    
import sys
import os

path = os.path.join('..', '/Users/dylandissanayake/Desktop/DPhil/Comp Disc/Repositories/TB-PNCA-GNN') if "SSH_CONNECTION" not in os.environ else os.path.join('..', '/mnt/alphafold-volume-1/dylan2/repos/tb-pnca-gnn')
if path not in sys.path:
    sys.path.append(os.path.abspath(path))

import datetime
import random

import numpy as np
import pandas as pd

import torch
from torch_geometric.data import Data

import wandb

import warnings
warnings.filterwarnings('ignore')

from src import run_model, protein_graph, gcn_model, evaluation

%load_ext autoreload
%autoreload 2

%aimport src

torch.cuda.is_available()

'Running via SSH'

/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.2.3) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


True

In [19]:
full_dataset = torch.load('datasets/full_real_dataset_v2.pth')

### Set Up Params and Sweep Config

In [3]:
seed = 42
np.random.seed(seed)
random.seed(seed)

# logging params (only used for wandb metrics)
n_samples = len(full_dataset)
cutoff_distance = 12  

# gcn params
num_node_features = 18
batch_size = 256
hidden_channels = 256
learning_rate = 1e-4
wd = 5e-6
epochs = 1500

# wt_seq = 'MRALIIVDVQNDFCEGGSLAVTGGAALARAISDYLAEAADYHHVVATKDFHIDPGDHFSGTPDYSSSWPPHCVSGTPGADFHPSLDTSAIEAVFYKGAYTGAYSGFEGVDENGTPLLNWLRQRGVDEVDVVGIATDHCVRQTAEDAVRNGLATRVLVDLTAGVSADTTVAALEEMRTASVELVCS'

In [33]:
# First sweep:

sweep_config = {
    'method': 'random'
    }

metric = {
    'name': 'Test Accuracy',
    'goal': 'maximize'   
    }

sweep_config['metric'] = metric

parameters_dict = {
    'hidden_channels': {
        'values': [32, 64, 128, 256]
        },
    'weight_decay': {
        'distribution': 'log_uniform_values',
        'min': 1e-8,
        'max': 1e-2
        },
    'dropout': {
          'values': [0.2, 0.4, 0.5, 0.6, 0.8]
        },
    'cutoff_distance': {
        'distribution': 'uniform',
        'min': 3.5,
        'max': 10 
        },
    'learning_rate': {
        'distribution': 'log_uniform_values',
        'min': 1e-8,
        'max': 1e-1
      },
    }

sweep_config['parameters'] = parameters_dict

# Second / third sweep:

parameters_dict.update({
    'hidden_channels': {
        'values': [64, 128, 192, 256, 320, 384]
        },
    'weight_decay': {
        'distribution': 'log_uniform_values',
        'min': 5e-7, 
        'max': 1e-2
        },
    'dropout': {
          'values': [0.4, 0.5, 0.6, 0.8]
        },
    'learning_rate': {
        'distribution': 'log_uniform_values',
        'min': 1e-5,
        'max': 1e-2 
        },
    })

# Fourth / Fifth sweep:

parameters_dict.update({
    'hidden_channels': {
        'values': [128, 192, 256, 320]
        },
    'weight_decay': {
        'distribution': 'log_uniform_values',
        'min': 1e-5,
        'max': 1e-3
        },
    'dropout': {
          'values': [0.4, 0.5, 0.6, 0.8]
        },
    'learning_rate': {
        'distribution': 'log_uniform_values',
        'min': 5e-5,
        'max': 5e-2
        },
    })

# Sixth sweep:

parameters_dict.update({
    'learning_rate': {
        'distribution': 'log_uniform_values',
        'min': 1e-6,
        'max': 5e-3 
        }
    })

# Seventh sweep:

parameters_dict.update({
    'learning_rate': {
        'distribution': 'log_uniform_values',
        'min': 5e-6,
        'max': 5e-4 
        }
    })

# Eight sweep w cutoff varying:

parameters_dict.update({
    'learning_rate': {
        'distribution': 'log_uniform_values',
        'min': 1e-15,
        'max': 1e-3 
        },
        'cutoff_distance': {
        'distribution': 'uniform',
        'min': 5.5,
        'max': 14
        },
    })



In [34]:
import pprint
pprint.pprint(sweep_config)

{'method': 'random',
 'metric': {'goal': 'maximize', 'name': 'Test Accuracy'},
 'parameters': {'cutoff_distance': {'distribution': 'uniform',
                                    'max': 14,
                                    'min': 5.5},
                'dropout': {'values': [0.4, 0.5, 0.6, 0.8]},
                'hidden_channels': {'values': [128, 192, 256, 320]},
                'learning_rate': {'distribution': 'log_uniform_values',
                                  'max': 0.001,
                                  'min': 1e-15},
                'weight_decay': {'distribution': 'log_uniform_values',
                                 'max': 0.001,
                                 'min': 1e-05}}}


### Define Training Loop

In [4]:
project = "pnca-singletons-sweep-v2"

In [ ]:
sweep_id = wandb.sweep(sweep_config, project=project)
# sweep_id = 'nc8eahq7'
# sweep_id = 'i8r2s0bc'

# sweep_id = 'env42e7l'

In [5]:
import pickle as pkl
with open('datasets/no_af_x18_exp_l2_graph_dict.pkl', 'rb') as f:
    graph_dict = pkl.load(f)
    
full_dataset_d = []
for sample in graph_dict['train']:
    full_dataset_d.append(graph_dict['train'][sample]['graph'].dataset[0])
for sample in graph_dict['test']:
    full_dataset_d.append(graph_dict['test'][sample]['graph'].dataset[0])

In [21]:
# Run a single run

model = run_model.pnca_simpleGCN(
            # sequences=sequences_dict,
            sequences=None,
            self_loops = False,
            cutoff_distance = cutoff_distance,
            edge_weight_func = 'exp',
            # edge_weight_func = 'none',
            batch_size = batch_size,
            # num_node_features = num_node_features,
            num_node_features = 16,
            hidden_channels = hidden_channels,
            learning_rate = learning_rate/10,
            wd = wd,
            dropout = 0.6,
            lr_scheduling=False,
            epochs = epochs,
            dataset = full_dataset,
            normalise_ews=True,
            lambda_param=2,
            early_stop=False,
            # wandb_params={
            #     'use_wandb' : True,
            #     'wandb_project' : project,
            #     # 'wandb_project' : 'pnca-sweep-1',
            #     'wandb_name' : 'rerun no af1',
            #     'n_samples' : n_samples
            # }
        )

Using CUDA
Epoch: 000, Train Acc: 0.5302, Test Acc: 0.4850, Train Loss: 0.6877, Test Loss: 0.6969
Epoch: 010, Train Acc: 0.5302, Test Acc: 0.4850, Train Loss: 0.6841, Test Loss: 0.6932
Epoch: 020, Train Acc: 0.5302, Test Acc: 0.5500, Train Loss: 0.6796, Test Loss: 0.6845
Epoch: 030, Train Acc: 0.5302, Test Acc: 0.6900, Train Loss: 0.6765, Test Loss: 0.6791
Epoch: 040, Train Acc: 0.5302, Test Acc: 0.6850, Train Loss: 0.6739, Test Loss: 0.6765
Epoch: 050, Train Acc: 0.5323, Test Acc: 0.5950, Train Loss: 0.6704, Test Loss: 0.6769
Epoch: 060, Train Acc: 0.5302, Test Acc: 0.7150, Train Loss: 0.6676, Test Loss: 0.6714
Epoch: 070, Train Acc: 0.5302, Test Acc: 0.6650, Train Loss: 0.6633, Test Loss: 0.6705
Epoch: 080, Train Acc: 0.5302, Test Acc: 0.6600, Train Loss: 0.6610, Test Loss: 0.6682
Epoch: 090, Train Acc: 0.5302, Test Acc: 0.6350, Train Loss: 0.6568, Test Loss: 0.6667
Epoch: 100, Train Acc: 0.6940, Test Acc: 0.5850, Train Loss: 0.6531, Test Loss: 0.6694
Epoch: 110, Train Acc: 0.6746, T

KeyboardInterrupt: 

In [21]:
def sweep_run():

    with wandb.init() as run:
        config = run.config

        model = run_model.pnca_simpleGCN(
            # sequences=sequences_dict,
            sequences=None,
            self_loops = False,
            cutoff_distance = config.cutoff_distance,
            edge_weight_func = '1-(dist/cutoff)',
            # edge_weight_func = 'none',
            batch_size = batch_size,
            num_node_features = num_node_features,
            hidden_channels = config.hidden_channels,
            learning_rate = config.learning_rate,
            wd = config.weight_decay,
            dropout = config.dropout,
            lr_scheduling=False,
            epochs = epochs,
            dataset = full_dataset,
            normalise_ews=True,
            wandb_params={
                'use_wandb': False,
                'sweep': True
            }
        )

        # os.makedirs(f'saved_models/carter_ds_aug/{project}/{sweep_id}', exist_ok=True)
        
        # torch.save(model, f'saved_models/carter_ds_aug/{project}/{sweep_id}/{run.name}')

In [22]:
wandb.agent(sweep_id, sweep_run, project = project, count=100)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


wandb: Ctrl + C detected. Stopping sweep.


In [7]:
torch.set_printoptions(precision=10)

In [8]:
graph_dict['train']['pnca_mut_0']['graph'].dataset[0].x[101]

tensor([0.4764460027, 0.0869565308, 0.3253291845, 0.3992491066, 0.9666666389,
        0.0000000000, 0.0000000000, 0.0000000000, 0.0961175859, 0.1682148278,
        0.8969879150, 0.0543248914, 0.0936825424, 0.0180733204, 0.8917525411,
        0.5736976862, 0.2387447655, 0.1145175770])

In [9]:
graph_dict['train']['pnca_mut_0']['graph'].dataset[0]

Data(x=[185, 18], edge_index=[2, 5264], edge_attr=[5264, 1], y=0)

In [12]:
full_dataset_d

[Data(x=[185, 18], edge_index=[2, 5264], edge_attr=[5264, 1], y=0),
 Data(x=[185, 18], edge_index=[2, 5264], edge_attr=[5264, 1], y=1),
 Data(x=[185, 18], edge_index=[2, 5264], edge_attr=[5264, 1], y=1),
 Data(x=[185, 18], edge_index=[2, 5264], edge_attr=[5264, 1], y=0),
 Data(x=[185, 18], edge_index=[2, 5264], edge_attr=[5264, 1], y=0),
 Data(x=[185, 18], edge_index=[2, 5264], edge_attr=[5264, 1], y=0),
 Data(x=[185, 18], edge_index=[2, 5264], edge_attr=[5264, 1], y=0),
 Data(x=[185, 18], edge_index=[2, 5264], edge_attr=[5264, 1], y=0),
 Data(x=[185, 18], edge_index=[2, 5264], edge_attr=[5264, 1], y=0),
 Data(x=[185, 18], edge_index=[2, 5264], edge_attr=[5264, 1], y=1),
 Data(x=[185, 18], edge_index=[2, 5264], edge_attr=[5264, 1], y=1),
 Data(x=[185, 18], edge_index=[2, 5264], edge_attr=[5264, 1], y=1),
 Data(x=[185, 18], edge_index=[2, 5264], edge_attr=[5264, 1], y=1),
 Data(x=[185, 18], edge_index=[2, 5264], edge_attr=[5264, 1], y=0),
 Data(x=[185, 18], edge_index=[2, 5264], edge_at

In [16]:
full_dataset

[Data(x=[185, 17], edge_index=[2, 5264], edge_attr=[5264, 1], y=0),
 Data(x=[185, 17], edge_index=[2, 5264], edge_attr=[5264, 1], y=0),
 Data(x=[185, 17], edge_index=[2, 5264], edge_attr=[5264, 1], y=1),
 Data(x=[185, 17], edge_index=[2, 5264], edge_attr=[5264, 1], y=0),
 Data(x=[185, 17], edge_index=[2, 5264], edge_attr=[5264, 1], y=1),
 Data(x=[185, 17], edge_index=[2, 5264], edge_attr=[5264, 1], y=1),
 Data(x=[185, 17], edge_index=[2, 5264], edge_attr=[5264, 1], y=1),
 Data(x=[185, 17], edge_index=[2, 5264], edge_attr=[5264, 1], y=1),
 Data(x=[185, 17], edge_index=[2, 5264], edge_attr=[5264, 1], y=0),
 Data(x=[185, 17], edge_index=[2, 5264], edge_attr=[5264, 1], y=0),
 Data(x=[185, 17], edge_index=[2, 5264], edge_attr=[5264, 1], y=1),
 Data(x=[185, 17], edge_index=[2, 5264], edge_attr=[5264, 1], y=0),
 Data(x=[185, 17], edge_index=[2, 5264], edge_attr=[5264, 1], y=1),
 Data(x=[185, 17], edge_index=[2, 5264], edge_attr=[5264, 1], y=0),
 Data(x=[185, 17], edge_index=[2, 5264], edge_at